In [0]:
import pandas as pd
import requests
import json
import openpyxl
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType

## 📄 Parameters

| Parameter | Type | Default | Description / Usage |
|-----------|------|---------|---------------------|
| `country` | string | `"it"` | Country code used to filter or customize execution. Currently only printed, but can be used for logic routing. |
| `topic` | string | `"dataverse"` | Data domain or topic (e.g., `"dataverse"`). Used to filter tables based on the `topic` column in the input CSV file. |
| `target_table_info_name` | string | `"table_info_dataverse.csv"` | Name of the CSV file containing target table information. Not directly loaded in this script (uses `dataverse_table_info.csv` hardcoded), but can be used in extensions. |
| `max_age` | int (seconds) | `86400` (24h) | Maximum allowed age (in seconds) since last modification of a table before it’s considered for refresh. If the table is “younger” than this limit, it will be refreshed. |
| `force_refresh_all` | boolean | `false` | If `true`, forces refresh of **all** tables regardless of `max_age` or row count checks. |
| `select_topics` | string (semicolon-separated list) | `None` | List of topics to include. If set, only tables with a `topic` value present in this list are processed. A trailing `;` is removed if present. |
| `force_fail` | boolean | `false` | If `true`, the notebook will raise an exception in case of an error when fetching Dataverse table metadata; otherwise, it will log the error and continue. |

---

### 🔑 Operational Notes
- Parameters are created using `dbutils.widgets` so they can be set via the Databricks UI or orchestrated from Azure Data Factory.
- **Azure Key Vault secrets** (`dataprocessing-tenant-id`, `dataprocessing-app-id`, `dataprocessing-app-secret`, `dataverse-uri`, etc.) are **used inside the notebooks** to authenticate with Dataverse. They are **not part of the ADF pipeline definition**.
- `select_topics` is normalized into a Python `set` for fast lookups.
- `max_age` is in **seconds** (`86400` = 24 hours).
- `force_refresh_all` and `force_fail` are dropdown boolean widgets in Databricks (`true` / `false`).



## 📚 Function Reference

This section describes the core utility functions used for:
- Tracking Dataverse table metadata (last update date & row count).
- Authenticating to the Dataverse API.
- Retrieving table status and counts.

---

### `get_stat_format(table_topic, table_name_plural, last_modified_dt, last_modified_count)`

**Purpose:**  
Returns a Python dictionary matching the `stats_tab_schema` format for easier DataFrame creation.

**Parameters:**
- `table_topic` *(str)* – Logical topic/group the table belongs to.
- `table_name_plural` *(str)* – Dataverse plural table name.
- `last_modified_dt` *(datetime)* – Last modification timestamp.
- `last_modified_count` *(int)* – Row count at last update.

**Returns:**  
A dictionary with keys:  
`table_topic`, `table_name_plural`, `last_modified_dt`, `last_modified_count`.

---

### `get_table_stats(table_name_plural)`

**Purpose:**  
Checks whether statistics for a given Dataverse table already exist in `<your_schema>.dataverse_update_lookup`.

**Parameters:**
- `table_name_plural` *(str)* – Dataverse plural table name.

**Returns:**  
Tuple:
1. `found?` *(bool)* – Whether an entry exists.
2. `count` *(int or None)* – Stored row count.
3. `last_dt` *(datetime or None)* – Stored last modification date.

---

### `update_table_stats(table_topic, table_name_plural, last_modified_dt, last_modified_count)`

**Purpose:**  
Updates or inserts statistics for a Dataverse table in `<your_schema>.dataverse_update_lookup`.

**Parameters:**
- `table_topic` *(str)*
- `table_name_plural` *(str)*
- `last_modified_dt` *(datetime)*
- `last_modified_count` *(int)*

**Notes:**
- If the record does not exist, it is **inserted**.
- If the record exists, the `last_modified_dt` and `last_modified_count` fields are **updated**.
- ⚠️ If two processes run the Dataverse lookup in parallel, they may both update the same table. This is not critical — at worst, the table will be refreshed twice.

---

### `get_access_token(tenant_id, client_id, client_secret, dataverse_uri)`

**Purpose:**  
Requests an OAuth 2.0 client credentials access token for the Dataverse API.

**Parameters:**
- `tenant_id` *(str)* – Azure AD tenant ID.
- `client_id` *(str)* – Application client ID.
- `client_secret` *(str)* – Application secret.
- `dataverse_uri` *(str)* – Dataverse instance URI (without `/api/data/v9.2/`).

**Returns:**  
Access token *(str)* to be used in HTTP `Authorization` headers.

**Raises:**  
Exception if the token request fails or the response does not contain `access_token`.

---

### `dataverse_count(table_name_plural, access_token)`

**Purpose:**  
Counts the number of rows in a Dataverse table by paging through results.

**Parameters:**
- `table_name_plural` *(str)* – Dataverse plural table name.
- `access_token` *(str)* – OAuth 2.0 access token.

**Returns:**  
Row count *(int)* for the specified table.

**Notes:**
- Uses `$select=modifiedon` and `$count=true` to minimize payload.
- Follows `@odata.nextLink` for pagination.

---

### `get_status_of_dataverse_table(table_name_plural, endpoint, access_token)`

**Purpose:**  
Retrieves the latest modification date and row count for a Dataverse table.

**Parameters:**
- `table_name_plural` *(str)* – Dataverse plural table name.
- `endpoint` *(str)* – Dataverse API endpoint (e.g., `https://<org>.crm.dynamics.com/api/data/v9.2`).
- `access_token` *(str)* – OAuth 2.0 access token.

**Returns:**  
Tuple:
1. `last_modified_date` *(datetime or None)* – Latest modification timestamp.
2. `row_count` *(int or None)* – Total number of rows.
3. `response` *(requests.Response)* – Original HTTP response object.

**Behavior:**
- Orders results by `modifiedon` descending and retrieves the most recent record.
- If no records exist, returns `datetime(1970, 1, 1, 0, 0, 1)` and count `0`.
- Calls `dataverse_count()` to get the full row count.

**Error Handling:**
- If the HTTP status code is not `200` or `202`, prints a warning and returns `(None, None, response)`.

---

### 🔑 Key Notes for GitHub Usage
- Replace `<your_schema>` with a schema name matching your Dataverse topic (e.g., `sch_statistics`).
- These functions are designed to be **idempotent** — running them multiple times will not cause data corruption.
- The statistics tracking table is critical for **incremental refresh logic** in the pipeline.


In [0]:
def get_stat_format(table_topic, table_name_plural, last_modified_dt, last_modified_count):
  return {
    "table_topic" : table_topic,
    "table_name_plural" : table_name_plural,
    "last_modified_dt" : last_modified_dt,
    "last_modified_count" : last_modified_count,
  }


def get_table_stats(table_name_plural):
  '''
  returns: found?, count, last_dt
  '''
  global df_stats
  data = df_stats[df_stats['table_name_plural'] == table_name_plural].collect()
  if len(data) == 0:
    return False, None, None
  return True, data[0]['last_modified_count'], data[0]['last_modified_dt']


def update_table_stats(table_topic, table_name_plural, last_modified_dt, last_modified_count):
  global df_stats, stats_tab_schema
  data = df_stats[df_stats['table_name_plural'] == table_name_plural].collect()
  if len(data) == 0:
    spark.createDataFrame(data=[get_stat_format(table_topic, table_name_plural, last_modified_dt, last_modified_count), ], schema=stats_tab_schema).write.insertInto("sch_statistics.dataverse_update_lookup")
  else:
    _ = spark.sql(f"""
    UPDATE sch_statistics.dataverse_update_lookup
    SET 
      last_modified_dt = to_timestamp('{last_modified_dt.strftime('%Y%m%d %H%M%S')}', 'yyyyMMdd HHmmss'),
      last_modified_count = {last_modified_count}
    WHERE  
      table_name_plural = '{table_name_plural}'
    """)


def get_access_token(tenant_id, client_id, client_secret, dataverse_uri):
    token_endpoint_url = (
        f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    )
    payload = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": f"{dataverse_uri}/.default", 
    }
    res = requests.post(token_endpoint_url, data=payload)
    if res.status_code != 200:
        raise Exception(f"ERROR while trying getting token from dataverse service! Reason:\n{res.content}")
    access_token = ''
    try:
        access_token = res.json()["access_token"]
    except Exception as e:
        raise Exception(f"ERROR while trying getting token from dataverse service! Reason:\n{e}")
    return access_token

def dataverse_count(table_name_plural, access_token):
  arg = f"{table_name_plural}?$select=modifiedon&$count=true"
  headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
    "Content-Type": "application/json; charset=utf-8"
  }

  count_val = 0
  url = f"{endpoint}/{arg}".replace(' ', '%20')
  while True:
    res = requests.get(url=url, headers=headers)
    count_val += len(res.json()['value'])

    if '@odata.nextLink' in res.json().keys():
      arg = res.json()['@odata.nextLink']
      url = f"{arg}".replace(' ', '%20')
    else:
      break
  
  return count_val

def get_status_of_dataverse_table(table_name_plural, endpoint, access_token):
    url = f"{endpoint}/{table_name_plural}?$count=true&$orderby=modifiedon desc&$top=1&$select=modifiedon".replace(' ', '%20')
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json",
        "Content-Type": "application/json; charset=utf-8"
    }
    res = requests.get(
        url=url, 
        headers=headers
    )

    if res.status_code not in (200, 202, ):
        print("WARNING:", res.status_code, "with content", res.content)
        return None, None, res

    res_json = res.json()['value']
    last_modified_date = datetime(1970, 1, 1, 0, 0, 1)
    if len(res_json) > 0:
        return datetime.fromisoformat(res_json[0]['modifiedon'].replace('Z', '')), dataverse_count(table_name_plural, access_token), res
    else:
        return last_modified_date, 0, res

In [0]:
dbutils.widgets.text( "topic", "","" ) 
topic = dbutils.widgets.get( "topic" ) or "dataverse"

dbutils.widgets.text( "target_table_info_name", "","" ) 
target_table_info_name = dbutils.widgets.get( "target_table_info_name" )

dbutils.widgets.text( "max_age", "","" ) # in seconds
max_age = int(dbutils.widgets.get( "max_age" ) or "86400") # DEFAULT: 24 hours

dbutils.widgets.dropdown('force_refresh_all', 'false', ['true', 'false'])
force_refresh_all = (dbutils.widgets.get( "force_refresh_all" ) or "false") == 'true'

dbutils.widgets.text( "select_topics", "","" )
select_topics = (dbutils.widgets.get( "select_topics" ) or '').strip()
select_topics = select_topics[:-1] if select_topics.endswith(';') else select_topics
if select_topics in ('', ';'):
    select_topics = None
else:
    select_topics = set(select_topics.split(';'))

dbutils.widgets.dropdown('force_fail', 'false', ['true', 'false'])
force_fail = (dbutils.widgets.get( "force_fail" ) or "false") == 'true'

env = dbutils.secrets.get(scope="kv-solution-01", key="environment")

env_mapping = {
    "development": "<your-dev-catalog>",
    "test": "<<your-dev-catalog>>",
    "acceptance": "<your-acceptance-catalog>",
    "production": "<your-production-catalog>"
}

uc_env = env_mapping[env]

print("environment:", env)
print("topic:", topic)
print("max_age:", max_age)
print("force_refresh_all:", force_refresh_all)
print("force_fail:", force_fail)
print("select_topics:", select_topics)
print("target_table_info_name:", target_table_info_name)

In [0]:
df = spark.read.format("csv").option("header", "true").option("encoding", "utf-8").load("file:/Workspace/Users/galimm02@heiway.net/dataverse_table_info.csv")

## 📊 Dataverse Table Statistics Management

This section of the notebook manages a **Delta table** used to track the last modification date and row count for Dataverse tables.  
It allows the pipeline to decide whether a table needs refreshing based on changes in Dataverse.

`<your_schema>` should be replaced with the schema corresponding to your data topic (e.g., sch_statistics, sch_sales, etc.).

### The statistics table:
`dataverse_update_lookup` is a Delta table used to store:
- **table_topic** – The logical topic/group the table belongs to.
- **table_name_plural** – The Dataverse API plural table name.
- **last_modified_count** – Total number of rows at last update.
- **last_modified_dt** – Timestamp of the most recent modification.

In [0]:
_ = spark.sql(f"CREATE SCHEMA IF NOT EXISTS sch_statistics;")

_ = spark.sql("""
CREATE TABLE IF NOT EXISTS sch_statistics.dataverse_update_lookup
(
  table_topic STRING,
  table_name_plural STRING, 
  last_modified_count INT,
  last_modified_dt TIMESTAMP
)
USING DELTA LOCATION 'dbfs:/mnt/dls/statistics/dataverse_update_lookup';
""")
df_stats = spark.sql(f"SELECT * FROM sch_statistics.dataverse_update_lookup")
stats_tab_schema = StructType([
  StructField("table_topic", StringType(), True),
  StructField("table_name_plural", StringType(), True),
  StructField("last_modified_count", IntegerType(), True),
  StructField("last_modified_dt", TimestampType(), True),
])

display(df_stats)

table_topic,table_name_plural,last_modified_count,last_modified_dt
spedito,hkibtmup_spedito_region_bu_local_ds_lookups,19,2025-06-16T12:23:46Z
spedito,hkibtmup_spedito_region_bu_local_id_lookups,19,2025-06-16T12:23:45Z
procurement,hki_procurement_suppliers_procurements,0,1970-01-01T00:00:01Z
service_level,hki_hkiservice_levelservice_levels,17,2025-02-19T13:36:35Z
stac,hki_mma_classificazione_materiale_famiglias,601,2025-01-27T16:10:17Z
spedito,hkibtmup_spedito_area_local_idlookups,12,2025-06-16T12:23:36Z
cutoff_dates,hki_cutoff_year_month_id_lookups,60,2025-03-31T16:28:48Z
service_level,hki_intervention_reason_ds_lookups,0,1970-01-01T00:00:01Z
spedito,hkibtmup_spedito_zone_mt_ds_lookups,28,2025-06-16T12:23:34Z
service_level,hki_intervention_state_ds_lookups,0,1970-01-01T00:00:01Z


In [0]:
dataprocessing_tenant_id = dbutils.secrets.get(scope="kv-solution-01",key="dataprocessing-tenant-id")
dataprocessing_client_id = dbutils.secrets.get(scope="kv-solution-01",key="dataprocessing-app-id")
dataprocessing_client_secret = dbutils.secrets.get(scope="kv-solution-01",key="dataprocessing-app-secret")
dataverse_uri = dbutils.secrets.get(scope="kv-solution-01",key="dataverse-uri").replace('api/data/v9.2/', '')
# dataverse_uri = 'https://hei-gis-itdatapr-d-eu-1.api.crm4.dynamics.com'

access_token = get_access_token(
    dataprocessing_tenant_id, 
    dataprocessing_client_id, 
    dataprocessing_client_secret, 
    dataverse_uri
)

endpoint = f"{dataverse_uri}/api/data/v9.2"
print("OK")

OK


## 🚀 Execution Logic – Selecting Tables to Refresh

This script is the **execution entry point** that:
1. Iterates over the list of Dataverse tables (`df`).
2. Checks their **last modified date** and **row count** against stored metadata.
3. Decides whether each table should be refreshed.
4. Produces:
   - `tables_to_refresh_list` – Python list with refresh metadata.
   - `table_info_output_csv` – CSV configuration for downstream processing.

In [0]:
now = datetime.now()
tables_to_refresh_list = list()
table_info_output_csv = "schema_name,topic,table_name,table_name_plural,key,refresh_mode,columns,current_date,last_modified_date,age\n"
for item in df.collect():
    if select_topics is not None and item['topic'] not in select_topics:
        print(f"Skipping table '{item['table_name']}' (not in topics list)")
        continue
    
    last_modified_date, last_modified_count, res = get_status_of_dataverse_table(item['table_name_plural'], endpoint, access_token)
    ds_found, ds_count, ds_last_dt = get_table_stats(item['table_name_plural'])

    if last_modified_date is None:
        msg = f"ERROR while checking last update for table '{item['table_name']}'"
        if force_fail:
            raise Exception(msg)
        else:
            print(msg)
            continue

    # CHECK 1: rows number
    check_rows = (not ds_found) or (ds_found and (last_modified_count != ds_count))

    # CHECK 2 last modified 
    age = now - last_modified_date
    check_last_modified_dt = age.total_seconds() <= max_age

    if check_rows or check_last_modified_dt or force_refresh_all:
        if force_refresh_all:
            print(f"FORCE REFRESH table '{item['table_name']}' to list of tables to refresh with last update: {last_modified_date} ({age})")
        else:
            print(f"Adding table '{item['table_name']}' to list of tables to refresh with last update: {last_modified_date} ({age})")
        
        tables_to_refresh_list.append({
            'schema_name' : item['schema_name'],
            'topic' : item['topic'],
            'table_name' : item['table_name'],
            'table_name_plural' : item['table_name_plural'],
            'key': item['key'],
            'refresh_mode' : item['refresh_mode'],
            'columns' : item['columns'],
            'current_date' : str(now),
            'last_modified_date' : str(last_modified_date),
            'age' : age.total_seconds()
        })

        table_info_output_csv += f"{item['schema_name']},{item['topic']},{item['table_name']},{item['table_name_plural']},{item['key']},{item['refresh_mode']},{item['columns']},{str(now)},{str(last_modified_date)},{age.total_seconds()}\n"

        update_table_stats(item['topic'], item['table_name_plural'], last_modified_date, last_modified_count)
    
    else:
        print(f"Skipping table '{item['table_name']}' with last update: {last_modified_date} ({age})")

FORCE REFRESH table 'hki_product_replenishment' to list of tables to refresh with last update: 2025-04-07 08:37:25 (126 days, 5:11:59.418564)
FORCE REFRESH table 'hki_d_product_lookup' to list of tables to refresh with last update: 2025-02-07 10:53:29 (185 days, 2:55:55.418564)


In [0]:
table_info_path = f"dbfs:/mnt/landing/{topic}"
table_info_file_path = f"dbfs:/mnt/landing/{topic}/{target_table_info_name}"
print(table_info_file_path)
print(table_info_path)
dbutils.fs.mkdirs(table_info_path)
dbutils.fs.rm(table_info_file_path)
dbutils.fs.put(table_info_file_path, contents=table_info_output_csv)

dbfs:/mnt/landing/it/dataverse/table_info_dataverse.csv
dbfs:/mnt/landing/it/dataverse
Wrote 240 bytes.


True

In [0]:
output_dict = {
    "tables" : tables_to_refresh_list,
    "table_info_name" : target_table_info_name
}

json_output = json.dumps(output_dict)

dbutils.notebook.exit(json_output)